### Gathering protein name/symbol data from KEGG database w/ API
> meant to be run in a folder with only this python script and the .csv files with geneID's we will be gathering the protein symbols from - nothing else. Will create new folder with the output (files w/ protein symbols). Gene ID's should be in the first column of original CSV files

In [ ]:
from Bio.KEGG import REST
import glob
import io
import os
import pandas as pd
import csv
import re
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

def to_df(result):
    """
    Turns tabulated data into pandas dataframe
        Will be used here to change entry taken from KEGG database into pandas dataframe
    """
    return pd.read_table(io.StringIO(result), header=None)

FileList = glob.glob('*.csv') #List of files we will be getting protein symbols from


for filename in FileList: #for every file we will be going through
    proteinSymbols = []
    df = pd.read_csv(filename)
    for identification in df[df.columns[0]]: #For every gene ID in the file
        k = REST.kegg_find("genes", identification).read() #query the database for the entry
        dataKegg = to_df(k) #transform into pandas dataframe
        proteinString = dataKegg[1][0] #get the protein symbol
        split_string = re.split(r'[;\ ,]', proteinString) #clean the protein symbol 
        if (split_string[0] == 'hypothetical'):
            proteinSymbols.append('Hypothetical Protein') #if no protein symbol exists
        else:
            proteinSymbols.append(split_string[0]) #take first protein symbol if there are multiple
    df['ProteinSymbol'] = proteinSymbols
    df.to_csv('proteinSymbols ' + filename)

In [113]:
import shutil
ProteinFiles = glob.glob('proteinSymbols *') #for every new proteinSymbol file made
if (not os.path.exists("Output - Protein symbols")): #create new folder for output
    path = os.path.join("Output - Protein symbols")
    os.mkdir(path, 0o666)
for file in ProteinFiles: #move the newly created files
    shutil.move(file, "Output - Protein symbols")

['proteinSymbols in_vitro_K10_vs_in_vitro_gc86 (1).csv.csv', 'proteinSymbols in_vivo_gc86_vs_in_vitro_gc86 (1).csv.csv', 'proteinSymbols in_vivo_K10_vs_in_vitro_K10 (1).csv.csv', 'proteinSymbols in_vivo_K10_vs_in_vivo_gc86 (1).csv.csv']
